# How to extract tweets from Twitter

Following is the Python code to extract tweets (using Twitter API, tweepy) with a specific keyword and save it as a CSV (Comma-Separated Value) file.

In [ ]:
import tweepy
import csv 
import time

You need to create an access token with a Twitter account. Then take the following 4 Values from OAuth setting: 
- consumer key
- consumer secret
- key
- secret

In [ ]:
consumer_key = 'ConsumerKeyString'
consumer_secret = 'ConsumerSecretString'
auth = tweepy.auth.OAuthHandler(consumer_key, consumer_secret, callback=None)

key = 'KeyString'
secret = 'SecretString'
auth.set_access_token(key, secret)

api = tweepy.API(auth)
csvFile = open('collectedTweets.csv','w', newline='')
csvWriter = csv.writer(csvFile)

c = tweepy.Cursor(api.search, 
                  q='keyword',
                  since='2017-05-01', 
                  until='2017-05-10', 
                  lang='en').items()

Twitter API has some restirictions for data collection on keyword search. The free keyword search is limited to 10 previous days, and getting access to the older tweets costs money. Moreover, the Twitter API has a limitation of 180 request within a 15 minutes interval per authenticated user. So, I use a 15-min sleep thread in my code.

In [ ]:
while True:
    try:
        tweet = c.next()
        if ((not tweet.retweeted) and ('RT @' not in tweet.text)):
            csvWriter.writerow([tweet.created_at, 
                                tweet.retweet_count, 
                                tweet.user.screen_name, 
                                tweet.user.followers_count,
                                tweet.user.location.encode('utf-8'), 
                                tweet.text.encode('utf-8')])
    except tweepy.TweepError:
        print('Goes to sleep!')
        time.sleep(60*15)
        print('Woke up!')
        continue
    except StopIteration:
        print('Done!')
        break

csvFile.close()